In [1]:
cd ..

/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling


/home/nicolas/anaconda3/envs/stable-dif/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# make sure you're logged in with \`huggingface-cli login\`
# import torch
# import argparse
import os
import torch
from torchvision.utils import make_grid, save_image
import numpy as np  
# from PIL import Image
import matplotlib.pyplot as plt
# import os
import lpips
from PIL import Image
import hydra
from hydra import initialize, compose
from omegaconf import DictConfig, OmegaConf
import torchvision.transforms as transforms 
from torchvision.datasets import VisionDataset
from torch.utils.data import DataLoader
from typing import Callable, Optional
# import glob
import torch.nn as nn

from datasets import build_loader

%load_ext autoreload
%autoreload 2

/home/nicolas/anaconda3/envs/stable-dif/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load config file
# with initialize(version_base=None, config_path="../_configs"):
#     cfg = compose(config_name='ddrmpp.yaml')

# # Define upsample operation
upsample = nn.Upsample(scale_factor=2, mode='nearest') 
# downsample = nn.AvgPool2d(2, 2)
# downsample = F.interpolate(scale_factor=0.5, mode='nearest')

# # Build loader
# loader = build_loader(cfg)
lpips_ = lpips.LPIPS(net='alex').cuda()

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/nicolas/anaconda3/envs/stable-dif/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nicolas/anaconda3/envs/stable-dif/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/nicolas/anaconda3/envs/stable-dif/lib/python3.12/site-packages/lpips/weights/v0.1/alex.pth


## REDDIF

In [21]:
# algo = 'dps'
# algo = 'rsd_stable'
# algo = 'rsd_stable_repulsion_gamma30'
algo = 'rsd_stable_nonaug'
max_img_id = '00077'

# img_path = f'/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/output/{algo}'
img_path = f'/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/output/{algo}/deblur_motion'
# img_path = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/eval/exp_hybryd/hybrid'
img_path_input = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/input/FFHQ'
coeff = 0.0
import os
import torch
from PIL import Image

dino = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')
dino = dino.eval()
# Calculate diversity
cnt = 0
mean_psnr = 0
mean_lpips = 0
for item in os.listdir(img_path_input):
    if item.split('.')[0] < max_img_id:
        # Check if the item is a directory (folder)
        img_psld_torch_list = []    
        for img_id in os.listdir(os.path.join(img_path, item.split('.')[0])):
            # if len(img_id.split('_')) == 3:
            # if img_id.split('_')[2] == "grid.png":
            #     continue
            img_id_path = os.path.join(img_path, item.split('.')[0], img_id)
            img = Image.open(img_id_path).convert('RGB')

            img_psld_torch = transforms.ToTensor()(img)
            img_psld_torch_list.append(img_psld_torch)
                # Transform to tensor the list
        img_psld_torch_list = torch.stack(img_psld_torch_list)
        # print(img_psld_torch_list.shape)
        # Compute similarity
        total_pair_wise_sim = 0.
        xo_list = dino(img_psld_torch_list)
        # del dino
        xo_list /= xo_list.norm(dim=-1, keepdim=True)
        # calculate the cosine similarity
        sim = (xo_list @ xo_list.T)
        # set the diagonal to be 0
        sim = sim - torch.diag(sim.diag())
        total_pair_wise_sim += sim.sum() / (sim.shape[0] * (sim.shape[0] - 1))
        torch.clear_autocast_cache()
        # print(total_pair_wise_sim)
                            
print(total_pair_wise_sim)

Using cache found in /home/nicolas/.cache/torch/hub/facebookresearch_dino_main


KeyboardInterrupt: 

In [5]:
1 - 0.9601

0.03990000000000005

In [33]:
# algo = 'reddiff'
algo = 'rsd_stable_nonaug'
gamma = 30
# algo = f'rsd_stable_repulsion_gamma{gamma}'
max_img_id = '00005'

# img_path = f'/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/output/{algo}'
# img_path = f'/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/output/{algo}/deblur_motion'
# img_path = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/eval/exp_hybryd/dps/inp_large_box'
# img_path = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/eval/exp_hybryd/rsld_gamma50/inp_large_box_gamma50'
img_path = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/datasets/images_appendix/rlsd/ablation_steps'
img_path_input = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/input/FFHQ'
# img_path_input = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/datasets/samples'
coeff = 0.0

if algo == 'rsd_stable' or algo=='reddiff' or algo == f'rsd_stable_repulsion_gamma{gamma}'  or algo == f'rsd_stable_repulsion_augm_gamma{gamma}' or algo == f'rsd_stable_nonaug':
    # Calculate FID score
    cnt = 0
    mean_psnr = 0
    mean_lpips = 0
    for item in os.listdir(img_path_input):
        if item.split('.')[0] == max_img_id:
            print(item)
            # Check if the item is a directory (folder)
            img_true_path = os.path.join(img_path_input, item)
            # print(item)
            img_true = Image.open(img_true_path).convert('RGB')
            img_true_torch = transforms.ToTensor()(img_true)
            img_true_torch = upsample(img_true_torch.unsqueeze(0)).squeeze()
            psnr_list = []
            lpips_list = []
            lpips_name_list = []
            for img_id in os.listdir(os.path.join(img_path, item.split('.')[0])):
                print(img_id)
                # if img_id.split('_')[2] == "grid.png":
                #     continue
                # if img_id.split('_')[2] > "2.png":
                #     continue
                # print(img_id)
                img_id_path = os.path.join(img_path, item.split('.')[0], img_id)
                img = Image.open(img_id_path).convert('RGB')

                img_psld_torch = transforms.ToTensor()(img)
                # print(img_psld_torch.shape)
                # img_psld_torch= nn.functional.interpolate(img_psld_torch.unsqueeze(0), size=(256, 256)).squeeze()

                mse = torch.mean((img_psld_torch - img_true_torch) ** 2, dim=(0, 1, 2))
                psnr = 10 * torch.log10(1 / (mse + 1e-10))
                psnr_list.append(psnr)
                print(psnr)

                lpips_val = lpips_(img_psld_torch.cuda(), img_true_torch.cuda())
                lpips_list.append(lpips_val)
                lpips_name_list.append(img_id)
                print(lpips_val)
                
                # mean_psnr = mean_psnr + psnr
                # mean_lpips = mean_lpips + lpips_val

                # cnt = cnt + 1

            print(f"Mean PSNR: {torch.max(torch.stack(psnr_list))}")
            mean_psnr = mean_psnr + torch.max(torch.stack(psnr_list))
            print(f"Mean LPIPS: {torch.min(torch.stack(lpips_list))}", lpips_name_list[torch.argmin(torch.stack(lpips_list))])
            mean_lpips = mean_lpips + torch.min(torch.stack(lpips_list))
            cnt = cnt + 1
            
            # Save for FID
            # img_max = lpips_name_list[torch.argmin(torch.stack(lpips_list))]
            # img_id_path = os.path.join(img_path, item.split('.')[0], img_max)
            # img_torch = Image.open(img_id_path).convert('RGB')
            # dest_folder = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/output/rsd_stable_nonaug/phase_retrieval/FID'
            # img_torch = transforms.ToTensor()(img_torch)
            # save_image(img_torch, f'{dest_folder}/{item}')
else:
    # Calculate FID score
    cnt = 0
    mean_psnr = 0
    mean_lpips = 0
    for item in os.listdir(img_path_input):
        if item.split('.')[0] < max_img_id:
            # print(item)
            # Check if the item is a directory (folder)
            img_true_path = os.path.join(img_path_input, item)
            # print(item)
            img_true = Image.open(img_true_path).convert('RGB')
            img_true_torch = transforms.ToTensor()(img_true)
            # img_true_torch = upsample(img_true_torch.unsqueeze(0)).squeeze()
            for img_id in os.listdir(os.path.join(img_path, item.split('.')[0])):
                if len(img_id.split('.')[0].split('_')) == 2:
                    print(img_id)
                    img_id_path = os.path.join(img_path, item.split('.')[0], img_id)
                    img = Image.open(img_id_path).convert('RGB')

                    img_psld_torch = transforms.ToTensor()(img)

                    mse = torch.mean((img_psld_torch - img_true_torch) ** 2, dim=(0, 1, 2))
                    psnr = 10 * torch.log10(1 / (mse + 1e-10))
                    # print(psnr)
                    mean_psnr = mean_psnr + psnr

                    lpips_val = lpips_(img_psld_torch.cuda(), img_true_torch.cuda())
                    # print(lpips_val)
                    mean_lpips = mean_lpips + lpips_val

                cnt = cnt + 1


print(mean_psnr/cnt)
print(mean_lpips/cnt)

00005.png
200.png
tensor(28.0846)
tensor([[[[0.0625]]]], device='cuda:0', grad_fn=<AddBackward0>)
100.png
tensor(28.2260)
tensor([[[[0.0648]]]], device='cuda:0', grad_fn=<AddBackward0>)
999.png
tensor(27.2656)
tensor([[[[0.0586]]]], device='cuda:0', grad_fn=<AddBackward0>)
300.png
tensor(26.6597)
tensor([[[[0.0621]]]], device='cuda:0', grad_fn=<AddBackward0>)
500.png
tensor(26.0531)
tensor([[[[0.0592]]]], device='cuda:0', grad_fn=<AddBackward0>)
Mean PSNR: 28.226043701171875
Mean LPIPS: 0.058589112013578415 999.png
tensor(28.2260)
tensor(0.0586, device='cuda:0', grad_fn=<DivBackward0>)


In [19]:
0.01/0.05

0.19999999999999998

# ONly for phase retrieval repulsipon

In [54]:
# algo = 'dps'
# algo = 'rsd_stable_nonaug'
gamma = 30
algo = f'rsd_stable_repulsion_augm_gamma{gamma}'
# algo = f'rsd_stable_repulsion_gamma{gamma}'
max_img_id = '00049'

# img_path = f'/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/output/{algo}'
img_path = f'/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/output/{algo}/hdr'
img_path_input = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/input/FFHQ'
# img_path_input = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/datasets/samples'
coeff = 0.0

if algo == 'rsd_stable' or algo == f'rsd_stable_repulsion_gamma{gamma}' or algo == f'rsd_stable_nonaug'or algo == f'rsd_stable_repulsion_augm_gamma{gamma}':
    # Calculate FID score
    cnt = 0
    mean_psnr = 0
    mean_lpips = 0
    for item in os.listdir(img_path_input):
        if item.split('.')[0] < max_img_id:
            print(item)
            # Check if the item is a directory (folder)
            img_true_path = os.path.join(img_path_input, item)
            # print(item)
            img_true = Image.open(img_true_path).convert('RGB')
            img_true_torch = transforms.ToTensor()(img_true)
            img_true_torch = upsample(img_true_torch.unsqueeze(0)).squeeze()
            psnr_list = []
            lpips_list = []
            psnr_name_list = []
            lpips_name_list = []
            for img_id in os.listdir(os.path.join(img_path, item.split('.')[0])):
                # print(img_id)
                # if img_id.split('_')[2] == "grid.png":
                #     continue
                # if img_id.split('_')[2] > "2.png":
                #     continue
                # if img_id.split('_')[1] == "hat":

                #     if img_id.split('_')[2] == "grid.png":
                #         continue
                    print(img_id)
                    img_id_path = os.path.join(img_path, item.split('.')[0], img_id)
                    img = Image.open(img_id_path).convert('RGB')

                    img_psld_torch = transforms.ToTensor()(img)
                    # print(img_psld_torch.shape)
                    # img_psld_torch= nn.functional.interpolate(img_psld_torch.unsqueeze(0), size=(256, 256)).squeeze()

                    mse = torch.mean((img_psld_torch - img_true_torch) ** 2, dim=(0, 1, 2))
                    psnr = 10 * torch.log10(1 / (mse + 1e-10))
                    psnr_list.append(psnr)
                    psnr_name_list.append(img_id)
                    # print(psnr)

                    lpips_val = lpips_(img_psld_torch.cuda(), img_true_torch.cuda())
                    lpips_list.append(lpips_val)
                    lpips_name_list.append(img_id)
                    # print(lpips_val)
                    
                    # mean_psnr = mean_psnr + psnr
                    # mean_lpips = mean_lpips + lpips_val

                # cnt = cnt + 1
            print(f"Mean PSNR: {torch.max(torch.stack(psnr_list))}", psnr_name_list[torch.argmax(torch.stack(psnr_list))])
            mean_psnr = mean_psnr + torch.max(torch.stack(psnr_list))
            print(f"Mean LPIPS: {torch.min(torch.stack(lpips_list))}", lpips_name_list[torch.argmin(torch.stack(lpips_list))])
            mean_lpips = mean_lpips + torch.min(torch.stack(lpips_list))
            cnt = cnt + 1

            # Save for FID
            img_max = lpips_name_list[torch.argmin(torch.stack(lpips_list))]
            img_id_path = os.path.join(img_path, item.split('.')[0], img_max)
            img_torch = Image.open(img_id_path).convert('RGB')
            dest_folder = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/output/rsd_stable_repulsion_augm_gamma30/phase_retrieval/FID'
            img_torch = transforms.ToTensor()(img_torch)
            save_image(img_torch, f'{dest_folder}/{item}')


print(mean_psnr/cnt)
print(mean_lpips/cnt)

00005.png
x_3.png
x_5.png
x_0.png
x_1.png
x_2.png
x_4.png
Mean PSNR: 29.541547775268555 x_5.png
Mean LPIPS: 0.2625907063484192 x_5.png
00009.png
x_3.png
x_5.png
x_0.png
x_1.png
x_2.png
x_4.png
Mean PSNR: 11.890018463134766 x_3.png
Mean LPIPS: 0.5719186067581177 x_3.png
00070.png
x_3.png
x_5.png
x_0.png
x_1.png
x_2.png
x_4.png
Mean PSNR: 15.43441390991211 x_4.png
Mean LPIPS: 0.5285373330116272 x_4.png
00015.png
x_3.png
x_5.png
x_0.png
x_1.png
x_2.png
x_4.png
Mean PSNR: 16.171236038208008 x_1.png
Mean LPIPS: 0.5994754433631897 x_1.png
00039.png
x_3.png
x_5.png
x_0.png
x_1.png
x_2.png
x_4.png
Mean PSNR: 18.824909210205078 x_3.png
Mean LPIPS: 0.4753820300102234 x_3.png
00019.png
x_3.png
x_5.png
x_0.png
x_1.png
x_2.png
x_4.png
Mean PSNR: 13.239730834960938 x_1.png
Mean LPIPS: 0.6138342022895813 x_1.png
00059.png
x_3.png
x_5.png
x_0.png
x_1.png
x_2.png
x_4.png
Mean PSNR: 26.070688247680664 x_2.png
Mean LPIPS: 0.38502591848373413 x_2.png
00094.png
x_3.png
x_5.png
x_0.png
x_1.png
x_2.png
x_4.p

In [ ]:
img_path = '/home/nzilberstein/Inverse/_exp/output_bip_small_nonaugm/RED_diff_nonaug'
it = 0
for item in os.listdir(img_path_input):
    # Check if the item is a directory (folder)
    if item.isnumeric():
        print(item)
        list_imgs = os.listdir(os.path.join(img_path_input, item))
        idx_img = np.random.choice(len(os.listdir(os.path.join(img_path_input, item))))
        img_true = Image.open(os.path.join(args.dir_path, item, list_imgs[idx_img])).convert('RGB')

        img_torch = transforms.ToTensor()(img_true)

        # Create folder if does not exist
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)

        save_image(img_torch, f'{dest_folder}/{it}.png')
    it = it + 1


In [20]:
# /home/nicolas/red_diff_stable/PSLD/stable-diffusion/outputs/psld-samples-bip/samples/00002.png
from PIL import Image
import lpips
# img_path = '/home/nzilberstein/Inverse/_exp/output_rip/PSLD/samples/00003/00000.png'
img_id_single = '00015'
img_path = f'/home/nzilberstein/Inverse/PSLD/stable-diffusion/outputs/psld-samples-bip/samples/{img_id_single}/00000.png'
# img_path = f'/home/nzilberstein/Inverse/_exp/output_rip/PSLD/samples_500_iter/{img_id_single}/00000.png'
img = Image.open(img_path).convert('RGB')

img_path_reddif = f'/home/nzilberstein/Inverse/_exp/output_rip/RED_diff/10.0/x_hat_1.png'
# img_path_reddif = f'/home/nzilberstein/Inverse/_exp/output_rip/RED_diff/{img_id_single}/0.0/x_hat_1.png'
img_reddif = Image.open(img_path_reddif).convert('RGB')

img_true_path = f'/home/nzilberstein/Inverse/_exp/input/FFHQ/{img_id_single}.png'
# img_true_path = '/home/nzilberstein/PSLD/diffusion-posterior-sampling/data/samples/00003.png'
img_true = Image.open(img_true_path).convert('RGB')

img_deg = Image.open('/home/nzilberstein/Inverse/_exp/output_inv/x_deg.png').convert('RGB')
# Define upsample operation
upsample = nn.Upsample(scale_factor=2, mode='nearest') 

FileNotFoundError: [Errno 2] No such file or directory: '/home/nzilberstein/Inverse/PSLD/stable-diffusion/outputs/psld-samples-bip/samples/00015/00000.png'

## 3D Generation

In [ ]:
# algo = 'reddiff'

# img_path = f'/home/nzilberstein/repository/constrained_sampling/_exp/output/{algo}'
img_path = '/home/nzilberstein/threestudio/outputs/dreamfusion-if/a_bulldozer_made_out_of_toy_bricks@20240621-234449/gamma200_false_seed35/save/it10000-test'
coeff = 0.0
import os
import torch
from PIL import Image

dino = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')
dino = dino.eval()
# Calculate diversity
cnt = 0
mean_psnr = 0
mean_lpips = 0
img_psld_torch_list = [] 
for img_id in os.listdir(img_path):
        # Check if the item is a directory (folder)
       
    img_id_path = os.path.join(img_path, img_id)
    img = Image.open(img_id_path).convert('RGB')

    img_psld_torch = transforms.ToTensor()(img)
    
    img_psld_torch_list.append(img_psld_torch[:, :, :512])
                # Transform to tensor the list
img_psld_torch_list = torch.stack(img_psld_torch_list, dim = 0)
# print(img_psld_torch_list.shape)
# Compute similarity
total_pair_wise_sim = 0.
xo_list = dino(img_psld_torch_list)
# del dino
xo_list /= xo_list.norm(dim=-1, keepdim=True)
# calculate the cosine similarity
sim = (xo_list @ xo_list.T)
# set the diagonal to be 0
sim = sim - torch.diag(sim.diag())
total_pair_wise_sim += sim.sum() / (sim.shape[0] * (sim.shape[0] - 1))
torch.clear_autocast_cache()
        # print(total_pair_wise_sim)
                            
print(total_pair_wise_sim)

In [ ]:
# save tensor as pdf
img_psld_torch_list[-1] 


grid = make_grid([img_psld_torch_list[6], img_psld_torch_list[-1]], nrow=4)
save_image(grid, 'bulldozer_seed35_nonrepul.pdf')